# Proyecto Nanotecnología.  

En el campo de la nanotecnología, los científicos realizan experimentos sobre materiales que operan a nano escala, estos generan datos relacionados con las propiedades físicas y químicas de los materiales estudiados.

Estos datos son importantes para comprender y predecir cómo los materiales se comportan al aplicar ciertas fuerzas o condiciones, como cambios de temperatura o presión. Para poder usarlos y poder realizar predicciones precisas, es necesario el construir una estructura de datos que permita el acceso de esa información de forma optimizada.

##Importamos los datos.

In [0]:
# creación de las carpetas "bronce", "silver", "gold". 

path = '/FileStore/tables'
dbutils.fs.mkdirs(path + "/bronce")
dbutils.fs.mkdirs(path + "/silver")
dbutils.fs.mkdirs(path + "/gold")

Out[79]: True

In [0]:
#Declaración de las variables src y dest
src1 = "dbfs:/FileStore/Materiales.csv"
dest1 = "dbfs:/FileStore/tables/bronce/Materiales.csv"

src2 = "dbfs:/FileStore/Condiciones.csv"
dest2 = "dbfs:/FileStore/tables/bronce/Condiciones.csv"

src3 = "dbfs:/FileStore/Propiedades.csv"
dest3 = "dbfs:/FileStore/tables/bronce/Propiedades.csv"


In [0]:
# Importación de las bibliotecas necesarias.
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr

# Creación de una instancia de SparkSession.
spark = SparkSession.builder.appName("CargaCSV").getOrCreate()

# Carga de los archivos CSV dentro de DataFrames.
df1 = spark.read.csv(src1, header=True, inferSchema=True)
df2 = spark.read.csv(src2, header=True, inferSchema=True)
df3 = spark.read.csv(src3, header=True, inferSchema=True)


In [0]:
#Observación de los datos.

df1.show(5) 
df2.show(5)  
df3.show(5) 

+-----------+--------------------+--------------------+--------------------+--------------+
|id_material|                name|   compuesto_quimico|          aplicacion|fecha_creacion|
+-----------+--------------------+--------------------+--------------------+--------------+
|          1|Nanopartículas de...|Nanopartículas de...|Revestimientos ce...|    2024-04-26|
|          2|Nanopartículas de...|Nanopartículas de...|              Imanes|    2024-02-02|
|          3|Nanopartículas de...|       MXenes (null)|Materiales de con...|    2024-07-05|
|          4|          Fullerenos|Nanopartículas de...|              Imanes|    2024-06-22|
|          5|Nanomateriales cr...|Nanopartículas de...|Pantallas de alta...|    2023-11-22|
+-----------+--------------------+--------------------+--------------------+--------------+
only showing top 5 rows

+--------------+-----------+------------------+----------------+--------------------+-----------------+
|id_experimento|id_material|temperatura_pru

##Limpieza de los datos

In [0]:
#unión de los tres DataFrames.   

df_union = df1.join(df2, "id_material").join(df3, "id_material")

In [0]:
#Datos repetidos por identificadores unicos. 

df_duplicates = df_union.groupBy("id_material", "id_experimento", "id_propiedad").count().filter("count > 1")
df_duplicates.show()


+-----------+--------------+------------+-----+
|id_material|id_experimento|id_propiedad|count|
+-----------+--------------+------------+-----+
+-----------+--------------+------------+-----+



In [0]:
#Datos duplicados por condiciones del Experimento.

df_duplicates = df_union.groupBy("id_material", "temperatura_prueba", "presion_aplicada", "duracion_experimento", "fecha_experimento").count().filter("count > 1")

df_duplicates.show() #Mostramos si existen datos duplicados.

+-----------+------------------+----------------+--------------------+-----------------+-----+
|id_material|temperatura_prueba|presion_aplicada|duracion_experimento|fecha_experimento|count|
+-----------+------------------+----------------+--------------------+-----------------+-----+
+-----------+------------------+----------------+--------------------+-----------------+-----+



In [0]:
#Datos duplicados por las propiedades del Material

df_duplicates = df_union.groupBy("id_material", "resistencia", "elasticidad", "conductividad_termica", "densidad", "diametro_nanoparticulas", "espesor_capa", "capacidad_absorcion_luz").count().filter("count > 1")

df_duplicates.show() #Mostramos si existen datos duplicados.

+-----------+-----------+-----------+---------------------+--------+-----------------------+------------+-----------------------+-----+
|id_material|resistencia|elasticidad|conductividad_termica|densidad|diametro_nanoparticulas|espesor_capa|capacidad_absorcion_luz|count|
+-----------+-----------+-----------+---------------------+--------+-----------------------+------------+-----------------------+-----+
+-----------+-----------+-----------+---------------------+--------+-----------------------+------------+-----------------------+-----+



In [0]:
# Contamos aquellos valores nulos en cada columna de df_union.
from pyspark.sql.functions import col, sum

df_union.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_union.columns]).show()

+-----------+----+-----------------+----------+--------------+--------------+------------------+----------------+--------------------+-----------------+------------+-----------+-----------+---------------------+--------+-----------------------+------------+-----------------------+
|id_material|name|compuesto_quimico|aplicacion|fecha_creacion|id_experimento|temperatura_prueba|presion_aplicada|duracion_experimento|fecha_experimento|id_propiedad|resistencia|elasticidad|conductividad_termica|densidad|diametro_nanoparticulas|espesor_capa|capacidad_absorcion_luz|
+-----------+----+-----------------+----------+--------------+--------------+------------------+----------------+--------------------+-----------------+------------+-----------+-----------+---------------------+--------+-----------------------+------------+-----------------------+
|          0|   0|                0|         0|             0|             0|                 0|               0|                   0|                0|  

In [0]:
#Cambiamos el nombre de la columna name a nombre. 

df_renamed = df_union.withColumnRenamed("name", "nombre")
df_renamed.describe() #Mostramos las columnas del dataframe. 

Out[88]: DataFrame[summary: string, id_material: string, nombre: string, compuesto_quimico: string, aplicacion: string, id_experimento: string, temperatura_prueba: string, presion_aplicada: string, duracion_experimento: string, id_propiedad: string, resistencia: string, elasticidad: string, conductividad_termica: string, densidad: string, diametro_nanoparticulas: string, espesor_capa: string, capacidad_absorcion_luz: string]

In [0]:
#Nos aseguramos de que nuestros datos tengan el tipo de dato correcto. 

from pyspark.sql.functions import col

df_corrected = (df_renamed
                .withColumn("id_material", col("id_material").cast("integer"))
                .withColumn("temperatura_prueba", col("temperatura_prueba").cast("integer"))
                .withColumn("presion_aplicada", col("presion_aplicada").cast("double"))
                .withColumn("duracion_experimento", col("duracion_experimento").cast("integer"))
                .withColumn("resistencia", col("resistencia").cast("integer"))
                .withColumn("elasticidad", col("elasticidad").cast("integer"))
                .withColumn("conductividad_termica", col("conductividad_termica").cast("double"))
                .withColumn("densidad", col("densidad").cast("double"))
                .withColumn("diametro_nanoparticulas", col("diametro_nanoparticulas").cast("integer"))
                .withColumn("espesor_capa", col("espesor_capa").cast("integer"))
                .withColumn("capacidad_absorcion_luz", col("capacidad_absorcion_luz").cast("integer"))
               )

df_corrected.printSchema() #Nos aseguramos que el tipo de datos sea el adecuado. 


root
 |-- id_material: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- compuesto_quimico: string (nullable = true)
 |-- aplicacion: string (nullable = true)
 |-- fecha_creacion: date (nullable = true)
 |-- id_experimento: integer (nullable = true)
 |-- temperatura_prueba: integer (nullable = true)
 |-- presion_aplicada: double (nullable = true)
 |-- duracion_experimento: integer (nullable = true)
 |-- fecha_experimento: date (nullable = true)
 |-- id_propiedad: integer (nullable = true)
 |-- resistencia: integer (nullable = true)
 |-- elasticidad: integer (nullable = true)
 |-- conductividad_termica: double (nullable = true)
 |-- densidad: double (nullable = true)
 |-- diametro_nanoparticulas: integer (nullable = true)
 |-- espesor_capa: integer (nullable = true)
 |-- capacidad_absorcion_luz: integer (nullable = true)



In [0]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import abs, udf, col
from pyspark.sql.types import ArrayType, DoubleType

# Convertimos los valores de 'duracion_experimento' a positivos
df_corrected = df_corrected.withColumn("duracion_experimento", abs(df_corrected["duracion_experimento"]))

# Seleccionamos las columnas numéricas para la normalización
numeric_cols = ["temperatura_prueba", "presion_aplicada", "duracion_experimento", "resistencia", "elasticidad", 
                "conductividad_termica", "densidad", "diametro_nanoparticulas", "espesor_capa", "capacidad_absorcion_luz"]

# Creamos un VectorAssembler para combinar las columnas numéricas en una sola columna de características
assembler = VectorAssembler(inputCols=numeric_cols, outputCol="features")

# Transformación del DataFrame
df_features = assembler.transform(df_corrected)

# Creamos un StandardScaler para normalizar las características sin restar la media
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=False, withStd=True)

# Ajustamos y transformamos los datos
scaler_model = scaler.fit(df_features)
df_scaled = scaler_model.transform(df_features)

# UDF para convertir el vector en una lista
vector_to_array = udf(lambda v: v.toArray().tolist(), ArrayType(DoubleType()))

# Convertimos la columna 'scaled_features' en una lista
df_array = df_scaled.withColumn("scaled_features_array", vector_to_array(col("scaled_features")))

# Seleccionar las columnas originales y las características normalizadas
df_final = df_array.select(
    ["id_material", "nombre", "compuesto_quimico", "aplicacion", "fecha_creacion", "id_experimento", "id_propiedad"] + 
    [col("scaled_features_array")[i].alias(numeric_cols[i]) for i in range(len(numeric_cols))]
)

# Mostrar las primeras filas para verificar
df_final.show(5)


+-----------+--------------------+--------------------+--------------------+--------------+--------------+------------+------------------+-----------------+--------------------+------------------+------------------+---------------------+------------------+-----------------------+------------------+-----------------------+
|id_material|              nombre|   compuesto_quimico|          aplicacion|fecha_creacion|id_experimento|id_propiedad|temperatura_prueba| presion_aplicada|duracion_experimento|       resistencia|       elasticidad|conductividad_termica|          densidad|diametro_nanoparticulas|      espesor_capa|capacidad_absorcion_luz|
+-----------+--------------------+--------------------+--------------------+--------------+--------------+------------+------------------+-----------------+--------------------+------------------+------------------+---------------------+------------------+-----------------------+------------------+-----------------------+
|          1|Nanopartículas 

In [0]:
# Creación de una vista temporal. 
df_final.createOrReplaceTempView("mi_vista_final")

In [0]:
%sql
SELECT * FROM mi_vista_final LIMIT 10;


id_material,nombre,compuesto_quimico,aplicacion,fecha_creacion,id_experimento,id_propiedad,temperatura_prueba,presion_aplicada,duracion_experimento,resistencia,elasticidad,conductividad_termica,densidad,diametro_nanoparticulas,espesor_capa,capacidad_absorcion_luz
1,Nanopartículas de óxido de samario,Nanopartículas de dióxido de titanio (TiO₂),Revestimientos cerámicos,2024-04-26,1,1,11.518931599429449,5.641146757237324,3.752820797450482,5.121737075643065,2.4145480829903834,1.8074487225380391,2.598593880128943,2.5799905949886974,2.011817808509382,7.679647279022113
2,Nanopartículas de óxido de disprosio,Nanopartículas de plata (Ag),Imanes,2024-02-02,2,2,9.486178964236016,4.773278025354659,3.5182694976098268,4.568035770168138,3.4709128692986764,1.7700532317269073,1.9904123337157862,2.364991378739639,1.9399671724911896,5.63174133794955
3,Nanopartículas de óxido de prometio,MXenes (null),Materiales de construcción,2024-07-05,3,3,10.502555281832732,6.075081123178657,3.2837181977691716,5.67543838111799,4.829096165980767,2.000658758395554,1.3822307873026294,1.3616617029107014,1.9399671724911896,6.911682551119902
4,Fullerenos,Nanopartículas de óxido de tungsteno (WO₃),Imanes,2024-06-22,4,4,10.841347387698304,6.942949855061322,2.8146155980878613,3.8759091383244813,6.036370207475959,0.891259197665309,3.4832215840026257,1.3616617029107014,0.8622076322183065,5.3757530953154795
5,Nanomateriales cristalinos,Nanopartículas de óxido de disprosio (Dy₂O₃),Pantallas de alta resolución,2023-11-22,5,5,10.841347387698304,5.207212391295991,4.691025996813102,4.429610443799407,3.3200036141117772,1.377400578210023,1.3269415558105242,2.436657784155992,1.2933114483274597,8.020964935867541
6,Nanopartículas de sílice,Nanopartículas de óxido de estaño (SnO₂),Conductores eléctricos,2024-09-17,6,6,11.85772370529502,7.376884221002654,2.110961698565896,2.906931853743361,5.432733186728362,1.763820649925052,3.151486195049995,2.0066593516578757,1.5807139924002285,6.2290472374290475
7,Nanopartículas de óxido de prometio,Nanodiamantes ©,Imanes,2024-08-06,7,7,10.16376317596716,6.942949855061322,2.8146155980878613,2.4916558746371664,5.432733186728362,3.7395490811131844,2.488015417144733,3.153321838319519,2.371070988600343,6.997011965331259
8,Nanopartículas de óxido de neodimio,Hidróxidos dobles laminares (null),Protección solar,2024-09-17,8,8,11.518931599429449,6.075081123178657,4.925577296653757,4.1527597910619445,4.5272776556069685,1.3898657418137337,2.875040037589469,1.0749960812452906,2.2992203525821506,7.850306107444827
9,Nanopartículas de óxido de tulio,Nanopartículas de platino (Pt),Imanes,2023-12-16,9,9,9.824971070101588,6.075081123178657,3.5182694976098268,4.1527597910619445,4.829096165980767,2.6862427565996376,2.6538831116210484,1.0749960812452906,1.5807139924002285,6.485035480063118
10,Nanopartículas de óxido de disprosio,Nanopartículas de óxido de samario (Sm₂O₃),Diagnóstico médico,2023-10-20,10,10,11.85772370529502,6.942949855061322,4.456474696972447,4.70646109653687,2.716366593364181,0.9535850156838621,2.488015417144733,3.2249882437358717,3.5925318009096103,7.082341379542616


In [0]:
# Define la ruta para la capa Silver
silver_path = "/mnt/delta/silver"

# Guarda el DataFrame como un archivo Delta
df_final.write.format("delta").mode("overwrite").save(silver_path)

# Registra la tabla en el catálogo de Databricks
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS silver_table
    USING DELTA
    LOCATION '{silver_path}'
""")

Out[100]: DataFrame[]

In [0]:
spark.sql("SELECT * FROM silver_table").show()

+-----------+--------------------+--------------------+--------------------+--------------+--------------+------------+------------------+-----------------+--------------------+------------------+------------------+---------------------+------------------+-----------------------+------------------+-----------------------+
|id_material|              nombre|   compuesto_quimico|          aplicacion|fecha_creacion|id_experimento|id_propiedad|temperatura_prueba| presion_aplicada|duracion_experimento|       resistencia|       elasticidad|conductividad_termica|          densidad|diametro_nanoparticulas|      espesor_capa|capacidad_absorcion_luz|
+-----------+--------------------+--------------------+--------------------+--------------+--------------+------------+------------------+-----------------+--------------------+------------------+------------------+---------------------+------------------+-----------------------+------------------+-----------------------+
|          1|Nanopartículas 